<a href="https://colab.research.google.com/github/shaye059/Machine-Learning-Stock-Prediction/blob/master/Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newsapi-python
!pip install -q wordcloud
!pip install yfinance

import wordcloud

from newsapi import NewsApiClient
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import sys
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import re

import pandas as pd
from datetime import date, timedelta
import yfinance as yf

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=5b8336f4a7c349803119594826b6557e4cffbd6b40573dee805afe820ece4db8
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


##Step 1 - Data extraction
Call the Google News API to get a list of articles for each company in a list. Since the model will be prediciting the effects of news stories, the companies should be as large as possible to ensure a greater variation in who holds stock, which in turn will mean a longer time for the news to spread to most shareholders. I'll also be limiting the companies to the tech sector to make it easier for the model to find trends in articles.

I've written the articles to a CSV file so that this portion does not need to be rerun unless you want to modify the companies or get an updated list of articles.

#### Don't run this section unless you modify the list of companies or want updated articles
I've saved the values into a CSV file that they can be read from.

If you do want to run this section you need a News API key.

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#

In [67]:

newsapi = NewsApiClient(api_key='') #Your Key Here

list_of_articles = []
dict_of_companies = {'Microsoft': 'MSFT','Apple':'AAPL','Amazon':'AMZN',
                     'Alphabet':'GOOGL','Facebook':'FB','Intel':'INTC',
                     'Cisco Systems':'CSCO','Comcast':'CMCSA','Adobe':'ADBE',
                     'Nvidia':'NVDA','Netflix':'NFLX','PayPal':'PYPL',
                     'Broadcom':'AVGO','Alibaba':'BABA','AT&T':'T',
                     'Tencent':'TCEHY','Taiwan Semiconductor':'TSM',
                     'Verizon':'VZ', 'Oracle':'ORCL','Tesla':'TSLA',
                     'Qualcomm':'QCOM','Texas Instruments':'TXN','Fiserv':'FISV',
                     'Booking Holdings':'BKNG','Intuit':'INTU','ADP':'ADP',
                     'T-Mobile':'TMUS','Micron':'MU','SAP':'SAP',
                     'Salesforce':'CRM','IBM':'IBM','VMware':'VMW',
                     'Samsung':'005930.KS','Foxconn':'2354.TW','Dell':'DELL',
                     'Sony':'SNE','Panasonic':'PCRFY','HP':'HP'}

# call the news api to retrieve a list of articles about Tesla
for company,ticker in dict_of_companies.items():
  temp = newsapi.get_everything(
                                        language='en', 
                                        q=company)['articles']
  for article in temp:
    article['company'] = company
    article['ticker'] = ticker
  list_of_articles = list_of_articles + temp


In [68]:
print(len(list_of_articles))
print(list_of_articles[0])
print(list_of_articles[-1])

760
{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Lauren Goode', 'title': 'Microsoft Duo: Price, Details, Release Date', 'description': 'Microsoft has had to shift its sales pitch from a “work” product to “staying home indefinitely” device.', 'url': 'https://www.wired.com/story/microsoft-surface-duo-price-release-date/', 'urlToImage': 'https://media.wired.com/photos/5f32e4cd37e06d513ea90690/191:100/w_1280,c_limit/Gear-Duo-Hero_01.jpg', 'publishedAt': '2020-08-12T13:00:00Z', 'content': 'The context of these kinds of devices has changed, says Ben Arnold, consumer technology analyst at the NPD Group, which tracks US sales of electronics. Its not the one-handed emailer on the subway an… [+2086 chars]', 'company': 'Microsoft', 'ticker': 'MSFT'}
{'source': {'id': 'engadget', 'name': 'Engadget'}, 'author': 'Valentina Palladino', 'title': 'The best affordable Windows laptops you can buy', 'description': 'If you’re a regular Engadget reader, you probably don’t think of cheap Windows l

In [69]:
df = pd.DataFrame(list_of_articles)
df.to_csv('News_Articles.csv', encoding='utf-8', index=False)

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#

### Reading from CSV
Read articles from CSV file into a Pandas Dataframe. I'm going to keep the title, description, and the date/time published so in the second line below I drop all other values for each article.

**NOTE: Before running this section you need to add the 'News_Articles.csv' file to the colab workspace. You can find the file on the project Github:** https://github.com/shaye059/Machine-Learning-Stock-Prediction

In [70]:
articles = pd.read_csv('News_Articles.csv')

articles.drop(articles.columns.difference(['title','description','publishedAt','company','ticker']), 1, inplace=True)


print(articles.iloc[[0, -1]])

                                               title  ... ticker
0        Microsoft Duo: Price, Details, Release Date  ...   MSFT
760  The best affordable Windows laptops you can buy  ...     HP

[2 rows x 5 columns]


## Step 2 - Preprocessing
Some simple POS lemmatization of the titles and descriptions to make it easier for the model to find trends.

*Note for later: There may be room to improve this by removing the companies name from the article. This may help if the model appears to be overfitting so I'll come back and change this based on performance.

In [71]:
## TODO: Lemmatize and POS


# helper function to tag words with their wordnet tag based on the nltk POS tag
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.ADV

In [72]:
wnl = nltk.WordNetLemmatizer()

def pos_lemmatize(string):
  # Cast to string to ensure all of them are
  string = str(string)
  
  # Tokenize
  tokens = nltk.word_tokenize(string.lower())

  # Keep only alpha numerics
  tokens_alpha = [t for t in tokens if re.match("^[a-zA-Z]+$", t)]

  # Remove stopwords
  tokens_alpha_stopless = [w for w in tokens_alpha if not w in stopwords.words('english')]

  # POS Tag
  pos_tokens = nltk.pos_tag(tokens_alpha_stopless)

  # POS-based lemmatization
  wordnet_tags = [get_wordnet_pos(p[1]) for p in pos_tokens]
  title_pos_lemmas = [wnl.lemmatize(t, w) for t, w in zip(tokens_alpha_stopless, wordnet_tags)]

  new_text = " ".join(title_pos_lemmas)

  return new_text

articles['title'] = articles['title'].apply(pos_lemmatize)
articles['description'] = articles['description'].apply(pos_lemmatize)

In [73]:
#pd.options.display.max_colwidth = 100

print(articles['title'].iloc[0:3])
print(articles['description'].iloc[0:3])

0              microsoft duo price detail release date
1    microsoft signal renew interest windows late r...
2    microsoft launch new family safety app io android
Name: title, dtype: object
0    microsoft shift sale pitch work product stay h...
1    microsoft reorganizing window team windows dev...
2    microsoft launch new family safety app io andr...
Name: description, dtype: object


Now get stock information from when each article was published to see its affect on the price. Start by comparing the opening price on the day the article was published to the closing price. If the article was published before/after trading hours, or on a non-trading day, then I'll look at the previous close and/or next open.

In [74]:
import math

temp_list = articles.to_dict('records')
temp_list2 = list(filter(lambda i: type(i['publishedAt']) != float, temp_list))
article_data = list(filter(lambda i: type(i['ticker']) != float, temp_list2))

for article in article_data:
  if not (type(article['publishedAt']) == float ):
    published = article['publishedAt']
    published_date = published[0:10]
    published_date = date(int(published_date[0:4]),int(published_date[5:7]),int(published_date[8:10]))
    
    # Check if the article was published on a trading day
    is_market_day = bool(len(pd.bdate_range(published_date, published_date)))
    
    # If it was a market day, see if it was published during trading hours
    if is_market_day:
      time = published[11:-1]

      # If the article was published before trading hours, compare the price from
      # the close of the previous day to the open of the current day.
      if time < '09:30:00':
        prev_day = published_date - timedelta(1)
        prev_market_day = bool(len(pd.bdate_range(prev_day, prev_day)))
        while not prev_market_day:
          prev_day = prev_day - timedelta(1)
          prev_market_day = bool(len(pd.bdate_range(prev_day, prev_day)))
        
        # A time delta of one day is added to each end day since yFinance only 
        # retrieves data upto and NOT including the end date.
        end = published_date + timedelta(1)

        article['start'], article['end'] = (prev_day, end)

      # The market typically closes at 16:00 (4pm) but the later limit
      # will be set to 15:30 (3:30pm) since most people will not have seen it
      # within the 30 minutes before closing. Compare close of current day to open
      # of next market day.
      elif time > '15:30:00':
        next_day = published_date + timedelta(1)
        next_market_day = bool(len(pd.bdate_range(next_day, next_day)))
        while not next_market_day:
          next_day = next_day + timedelta(1)
          next_market_day = bool(len(pd.bdate_range(next_day, next_day)))
        end = next_day + timedelta(1)
        article['start'], article['end'] = published_date, end

      # The article was published during market hours so compare opening of
      # current day to close of current day
      else:
        article['start'] = published_date
        article['end'] = published_date + timedelta(1)
    
    # The article was posted on a day that the market was closed so compare the
    # close of the last market day to the opening of the next.
    else:
      prev_day = published_date - timedelta(1)
      prev_market_day = bool(len(pd.bdate_range(prev_day, prev_day)))
      next_day = published_date + timedelta(1)
      next_market_day = bool(len(pd.bdate_range(next_day, next_day)))
      while not prev_market_day:
        prev_day = prev_day - timedelta(1)
        prev_market_day = bool(len(pd.bdate_range(prev_day, prev_day)))
      while not next_market_day:
          next_day = next_day + timedelta(1)
          next_market_day = bool(len(pd.bdate_range(next_day, next_day)))
      article['start'] = prev_day
      article['end'] = next_day + timedelta(1)


In [75]:
for article in article_data[0:3]:
  print('Start: ' + str(article['start']) + ' End: ' + str(article['end']))

Start: 2020-08-12 End: 2020-08-13
Start: 2020-08-07 End: 2020-08-08
Start: 2020-07-28 End: 2020-07-30


#### Ticker Data Retrieval
This first cell retrieves the data from yFinance. If an error occurs while running the second cell below, try rerunning the first.

In [76]:
# Ticker symbol for the company we want
tickers = []


# This could iterate through dict_of_companies, will seperate that out and chage 
# this part later
for article in article_data:
  if not article['ticker'] in tickers:
    tickers.append(article['ticker'])

tickerData = {}
for ticker in tickers:

  tickerData[ticker] = yf.Ticker(ticker)
#tickerSymbol = 'AMZN'

#tickerData = yf.Ticker(tickerSymbol)

In [77]:
## TODO: Save the delta for each article. Possibly remove start, end, and
##       publishedAt from each one so that there's only the necessary data.
for article in article_data:

  try:
    tickeramzn = tickerData[article['ticker']].history(interval='1d', start=str(article['start']), end=str(article['end']))
    if len(tickeramzn) == 1:
      day = tickeramzn.iloc[0]
      delta = (day['Close'] - day['Open']) / day['Open'] *100
      article['Delta'] = delta

    if len(tickeramzn) == 2:
      day1 = tickeramzn.iloc[0]
      day2 = tickeramzn.iloc[1]
      delta = (day2['Open'] - day1['Close']) / day1['Close'] *100
      article['Delta'] = delta
  except:
    print('Error retrieving values from yFinance, try running first cell again')

In [78]:
for article in article_data[0:3]:
  print(article['Delta'])

1.8997515709484172
-1.1030951826855968
0.23760023760023252
